For baseline solution I used simple tf-idf model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from tqdm.notebook import tqdm

from warnings import filterwarnings
filterwarnings("ignore")

## Load data

In [2]:
train = pd.read_csv("data/HeadHunter_train.csv")
test = pd.read_csv("data/HeadHunter_test.csv")
sample_submission = pd.read_csv("data/HeadHunter_sample_submit.csv")

positive_sentences = np.load("data/positive_sentences.npy")
negative_sentences = np.load("data/negative_sentences.npy")

print(f"Train shape: {train.shape} | Test shape: {test.shape}")

Train shape: (50876, 12) | Test shape: (50651, 11)


## Model

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from shared import calculate_metrics

import optuna

In [4]:
# data
text = [positive_sentences[i] + " " + negative_sentences[i] for i in range(len(positive_sentences))]
text = np.array(text)

# target
train.dropna(subset=["positive", "negative"], inplace=True, how="all")
train["preprocessed_target"] = train["target"].str.split(",").apply(lambda x: x[0]).astype(int)
target = train["preprocessed_target"].values

print(f"Input shape: {text.shape}, Target shape : {target.shape}")

Input shape: (50837,), Target shape : (50837,)


In [5]:
# split
text_train, text_test, target_train, target_test = train_test_split(text, target)
print(f"Train shape: {text_train.shape}, Test shape: {text_test.shape}")

Train shape: (38127,), Test shape: (12710,)


In [7]:
def objective(trial):

    # params
    C = trial.suggest_float("C", 1e-10, 3)
    max_df = trial.suggest_int("max_df", 1, 5)
    ngram_range = trial.suggest_int("ngram_range", 1, 5)
    
    # train
    pipeline = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,ngram_range), max_df=max_df)),
                         ("clf", OneVsRestClassifier(LogisticRegression(max_iter=1000, C=C)))])
    pipeline.fit(text_train, target_train)
    
    # metrics
    preds = pipeline.predict(text_test)
    acc, pr, rc = calculate_metrics(y_true=target_test, y_pred=preds)
    
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2022-01-24 20:32:24,463] A new study created in memory with name: no-name-9731fba2-bbbd-48f3-9b24-b8122661f2db
[I 2022-01-24 20:33:43,140] Trial 0 finished with value: 0.611 and parameters: {'C': 2.2209704003741435, 'max_df': 2, 'ngram_range': 5}. Best is trial 0 with value: 0.611.


Accuracy: 0.611, Precision: 0.314, Recall: 0.15


[I 2022-01-24 20:34:23,246] Trial 1 finished with value: 0.479 and parameters: {'C': 0.26351563639139375, 'max_df': 3, 'ngram_range': 4}. Best is trial 0 with value: 0.611.


Accuracy: 0.479, Precision: 0.157, Recall: 0.112


[I 2022-01-24 20:35:13,209] Trial 2 finished with value: 0.596 and parameters: {'C': 1.3473065154012838, 'max_df': 2, 'ngram_range': 4}. Best is trial 0 with value: 0.611.


...

Accuracy: 0.641, Precision: 0.26, Recall: 0.159


[I 2022-01-24 20:50:05,584] Trial 28 finished with value: 0.656 and parameters: {'C': 2.3403352359884884, 'max_df': 5, 'ngram_range': 2}. Best is trial 13 with value: 0.665.


Accuracy: 0.656, Precision: 0.29, Recall: 0.166


[I 2022-01-24 20:51:07,010] Trial 29 finished with value: 0.656 and parameters: {'C': 2.202621156239747, 'max_df': 4, 'ngram_range': 4}. Best is trial 13 with value: 0.665.


Accuracy: 0.656, Precision: 0.358, Recall: 0.165


In [8]:
study.best_params

{'C': 1.810072362948492, 'max_df': 5, 'ngram_range': 3}

In [12]:
%%time
pipeline = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,study.best_params["ngram_range"]),
                                               max_df=study.best_params["max_df"])),
                     ("clf", OneVsRestClassifier(LogisticRegression(max_iter=1000,
                                                                    C=study.best_params["C"])))])
pipeline.fit(text_train, target_train)
preds = pipeline.predict(text_test)
acc, pr, rc = calculate_metrics(y_true=target_test, y_pred=preds)

Accuracy: 0.665, Precision: 0.373, Recall: 0.167
CPU times: user 2min 27s, sys: 948 ms, total: 2min 28s
Wall time: 33.3 s


In [13]:
# check distribution
display(np.unique(preds, return_counts=True))
display(np.unique(target_test, return_counts=True))

(array([0, 1, 3, 8]), array([4752,    1,    6, 7951]))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([5282,  765,    1,  263,   23,   61,  194,   95, 6026]))